In [1]:
import os
import requests
from pydantic import BaseModel, ValidationError
from typing import Annotated, Literal, Optional
from datetime import datetime
from autogen import ConversableAgent, register_function
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.embeddings import HuggingFaceEmbeddings

load_dotenv()

True

In [2]:
# %pip install -qU langchain-openai
# !pip install spacy

In [3]:
# !pip install --upgrade langchain


In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chains import LLMChain
import numpy as np
import pandas as pd
from io import StringIO
import spacy
from bs4 import BeautifulSoup
import time
import json
import re
from typing import List, Dict, Annotated

In [5]:
FMP_API_KEY = os.getenv("FMP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [6]:
# Define the input model for tools
class StandardInput(BaseModel):
    ticker: Annotated[str, "The stock ticker symbol (e.g., AAPL, MSFT)."]
    date_range: Annotated[list[str], f"An array containing the start and end dates for the data needed in YYYY-MM-DD format. If not provided, provide a fitting range based on the prompt with the knowledge that today's date is {datetime.now().strftime('%Y-%m-%d')}. Try to be as current as possible."]

# Define new input model for tools if they dont follow the structure of StandardInput model or else use the same.

class PromptInput(BaseModel):
    prompt: Annotated[str, "The user's query or prompt."]

# Define the output model for the category
class CategoryOutput(BaseModel):
    category: Annotated[str, "The category of the financial instrument (stock, forex, crypto, other)."]

# Deine the input model for sec insights
class QualitativeDataInput(BaseModel):
    user_query: Annotated[str, "User query as it is"] = None
    ticker: Annotated[str, "The stock ticker symbol (e.g., AAPL, MSFT)."] = None
    document_type: Annotated[Optional[str], "The document type can either be 'Form 10K' or 'Form 10Q'."]
    date_range: Annotated[Optional[list[str]], f"An array containing the start and end dates for the data needed in YYYY-MM-DD format. If not provided, provide a fitting range for the most recently completed financial year based on the prompt, considering today's date is {datetime.now().strftime('%Y-%m-%d')}. Ensure the dates are as current as possible."]
    

# Deine the input model for sec insights
class SecInsightsInput(BaseModel):
    ticker: Annotated[str, "The stock ticker symbol (e.g., AAPL, MSFT)."]
    document_type: Annotated[str, "The document type can either be 'Form 10K' or 'Form 10Q'."]
    year: Annotated[str, "The year if 'Form 10K' is chosen (e.g., 2023,2001) or else it is the specific quarter of the year if 'Form 10Q' is chosen (e.g., 2015 Q2, 2020 Q3)."]

# Define the input model to create a tool that creates tools.
class MasterToolInput(BaseModel):
    url: Annotated[str, "The fmp url for a particular section for example #stocks, #forex etc."]
    

In [7]:
# function to create embedding
# model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
#                                     model_kwargs={'device': 'cpu'})
# embed_querry = model.embed_query("shares")

# def get_embedding(text,model):
#     return model.embed_query(text)

def get_embedding(text,tokenizer,model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy().reshape(1, -1)


def categorize_prompt_with_embeddings(prompt: str) -> str:

    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    model = AutoModel.from_pretrained("distilbert-base-uncased")
    
    # Define lists of keywords for each category
    stock_keywords = ["stock", "shares","share price", "AAPL", "MSFT", "equity", "NASDAQ", "NYSE", "ticker"]
    forex_keywords = ["forex", "currency", "exchange rate", "FX", "EUR/USD", "GBP/USD"]
    crypto_keywords = ["crypto", "cryptocurrency", "Bitcoin", "Ethereum", "BTC", "ETH"]

    # create embeddings for each of the keywords
    stock_embeddings = [get_embedding(kw,tokenizer,model) for kw in stock_keywords]
    forex_embeddings= [get_embedding(kw,tokenizer,model) for kw in forex_keywords]
    crypto_embeddings = [get_embedding(kw,tokenizer,model) for kw in crypto_keywords]

    # prompt embedding
    prompt_embedding = get_embedding(prompt,tokenizer,model)

    # metric 1
    stock_similarity = max(cosine_similarity(prompt_embedding,stock_emb)[0][0] for stock_emb in stock_embeddings)
    forex_similarity = max(cosine_similarity(prompt_embedding,forex_emb)[0][0] for forex_emb in forex_embeddings)
    crypto_similarity = max(cosine_similarity(prompt_embedding,crypto_emb)[0][0] for crypto_emb in crypto_embeddings)

    #metric 2
    stock_euclidean = min([np.linalg.norm(prompt_embedding - emb) for emb in stock_embeddings])
    forex_euclidean = min([np.linalg.norm(prompt_embedding - emb) for emb in forex_embeddings])
    crypto_euclidean = min([np.linalg.norm(prompt_embedding - emb) for emb in crypto_embeddings])

    # Ensembling Metrics
    stock_final_similarity = (stock_similarity + 1/stock_euclidean) / 2
    forex_final_similarity = (forex_similarity + 1/forex_euclidean) / 2
    crypto_final_similarity = (crypto_similarity + 1/crypto_euclidean) / 2

    # Make a decision based on the highest combined similarity
    similarities = {
        "stock": stock_final_similarity,
        "forex": forex_final_similarity,
        "crypto": crypto_final_similarity
    }


    category = max(similarities, key=similarities.get)
    
    return category

def fetch_symbols():
    # Fetching stock symbols
    stock_symbols = requests.get(f'https://financialmodelingprep.com/api/v3/stock/list?apikey={FMP_API_KEY}').json()
    stock_symbols = [item['name'] for item in stock_symbols] + [item['symbol'] for item in stock_symbols]
    
    # Fetching forex pairs
    forex_pairs = requests.get(f'https://financialmodelingprep.com/api/v3/forex?apikey={FMP_API_KEY}').json()
    forex_symbols = [item['ticker'] for item in forex_pairs['forexList']]
    
    # Fetching crypto symbols
    crypto_symbols = requests.get(f'https://financialmodelingprep.com/api/v3/symbol/available-cryptocurrencies?apikey={FMP_API_KEY}').json()
    crypto_symbols = [item['symbol'] for item in crypto_symbols] + [item['name'] for item in crypto_symbols]

    return stock_symbols, forex_symbols, crypto_symbols

def extract_keywords(prompt):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(prompt)
    keywords = [token.text.lower() for token in doc if not token.is_stop and token.is_alpha]
    return keywords

def identify_context(keywords,crypto_keywords,stock_keywords,forex_keywords):
    if any(keyword in crypto_keywords for keyword in keywords):
        return 'Crypto'
    elif any(keyword in stock_keywords for keyword in keywords):
        return 'Stock'
    elif any(keyword in forex_keywords for keyword in keywords):
        return 'Forex'
    return 'Unknown'

def scrape_keywords(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract keywords - This example assumes the keywords are in <li> elements
    keywords = [item.text for item in soup.find_all('li')]
    return keywords

def refine_keywords(keywords):
    nlp = spacy.load("en_core_web_sm")
    refined_keywords = []
    for keyword in keywords:
        doc = nlp(keyword)
        for token in doc:
            if not token.is_stop and token.is_alpha:
                refined_keywords.append(token.text.lower())
    return list(set(refined_keywords))

def entity_recognition(input):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(input)
    entities = []
    # Print the named entities found in the text
    for ent in doc.ents:
        entities.append(ent.text)

    return entities
    

In [8]:
print(extract_keywords("What is the current value of Bitcoin"))
print(refine_keywords("What is the current value of Bitcoin"))

['current', 'value', 'bitcoin']
['o', 'h', 't', 'w', 'u', 'r', 'v', 'c', 's', 'l', 'f', 'b', 'e', 'n']


In [9]:
import spacy

# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

# Process the sentence
text = "What is the value of JPY?"
doc = nlp(text)

# Print the named entities found in the text
for ent in doc.ents:
    print(ent.text, ent.label_)

print(entity_recognition("What is the current value of US Dollars (USD)?"))

JPY ORG
['US Dollars']


In [10]:
keywords_file = 'test.json'

def load_keywords(file_path):
    """Load keywords from a JSON file."""
    with open(file_path, 'r') as file:
        return json.load(file)

def save_keywords(file_path,keywords):
    """Save keywords back to the JSON file."""
    with open(file_path, 'w') as file:
        json.dump(keywords, file, indent=4)

def add_keyword(file_path,category, keyword):
    """Add a keyword to the specified category and update the file."""
    keywords = load_keywords(file_path)
    if keyword not in keywords[category]:
        keywords[category].append(keyword)
        save_keywords(file_path,keywords)
        return True
    return False



In [11]:
# text = "How is the market reacting to the latest DeFi hacks?"
# keywords = []
# entity = entity_recognition(text)
# entity

# if entity == []:
#     keywords.extend()

In [12]:
def categorize_crypto_forex_stock_tool_v4(input: Annotated[str, "Input prompt for classification"]) -> CategoryOutput:
    pass

    

In [13]:

stock_url2 = 'https://www.investopedia.com/financial-term-dictionary-4769738'
crypto_url2 = 'https://www.coindesk.com/learn/crypto-dictionary'
forex_url2 = 'https://www.babypips.com/forexpedia'

stock_keywords = scrape_keywords(stock_url2)
crypto_keywords = scrape_keywords(crypto_url2)
forex_keywords = scrape_keywords(forex_url2)

print("Stock Keywords:", stock_keywords)
print("Crypto Keywords:", crypto_keywords)
print("Forex Keywords:", forex_keywords)

final_stock_keywords_refined = refine_keywords(stock_keywords)
final_crypto_keywords_refined = refine_keywords(crypto_keywords)
final_forex_keywords_refined = refine_keywords(forex_keywords)

print("Refined Stock Keywords:", final_stock_keywords_refined)
print("Refined Crypto Keywords:", final_crypto_keywords_refined)
print("Refined Forex Keywords:", final_forex_keywords_refined)




Stock Keywords: ['\n Investing\n\n\n\n Stocks\n \n\n Cryptocurrency\n \n\n Bonds\n \n\n ETFs\n \n\n Options and Derivatives\n \n\n Commodities\n \n\n Trading\n \n\n Automated Investing\n \n\n Brokers\n \n\n Fundamental Analysis\n \n\n Markets\n \n\n View All\n \n\n', '\n Stocks\n ', '\n Cryptocurrency\n ', '\n Bonds\n ', '\n ETFs\n ', '\n Options and Derivatives\n ', '\n Commodities\n ', '\n Trading\n ', '\n Automated Investing\n ', '\n Brokers\n ', '\n Fundamental Analysis\n ', '\n Markets\n ', '\n View All\n ', '\n Simulator\n\n\n\n Login / Portfolio\n \n\n Trade\n \n\n Research\n \n\n My Games\n \n\n Leaderboard\n \n\n', '\n Login / Portfolio\n ', '\n Trade\n ', '\n Research\n ', '\n My Games\n ', '\n Leaderboard\n ', '\n Banking\n\n\n\n Savings Accounts\n \n\n Certificates of Deposit (CDs)\n \n\n Money Market Accounts\n \n\n Checking Accounts\n \n\n View All\n \n\n', '\n Savings Accounts\n ', '\n Certificates of Deposit (CDs)\n ', '\n Money Market Accounts\n ', '\n Checking Account

In [14]:
# Define the tool function
def upgrades_downgrades_analysis_tool(input: Annotated[StandardInput, "Input parameters for retrieving upgrades and downgrades data"]) -> str:
    ticker = input.ticker
    start_date = input.date_range[0]
    end_date = input.date_range[1]

    # Upgrades & Downgrades API
    upgrades_downgrades_url = f"https://financialmodelingprep.com/api/v4/upgrades-downgrades?symbol={ticker}&apikey={FMP_API_KEY}"
    upgrades_downgrades_response = requests.get(upgrades_downgrades_url)
    upgrades_downgrades_data = upgrades_downgrades_response.json()

    # Filter upgrades and downgrades data based on the date range
    filtered_upgrades_downgrades_data = [
        item for item in upgrades_downgrades_data
        if start_date <= item['publishedDate'][:10] <= end_date
    ]

    # Sort the filtered data by date in descending order
    sorted_upgrades_downgrades_data = sorted(filtered_upgrades_downgrades_data, key=lambda x: x['publishedDate'], reverse=True)

    # Upgrades & Downgrades Consensus API
    consensus_url = f"https://financialmodelingprep.com/api/v4/upgrades-downgrades-consensus?symbol={ticker}&apikey={FMP_API_KEY}"
    consensus_response = requests.get(consensus_url)
    consensus_data = consensus_response.json()

    # Combine the data into a single string
    analysis = f"Stock Ticker: {ticker}\n\n"

    # Handle Upgrades & Downgrades data dynamically
    if sorted_upgrades_downgrades_data is not None and len(sorted_upgrades_downgrades_data) > 0:
        analysis += "\nUpgrades & Downgrades (Most Recent):\n"
        token_count = 0
        for item in sorted_upgrades_downgrades_data:
            item_analysis = ""
            for key, value in item.items():
                item_analysis += f"{key}: {value}\n"
            item_analysis += "\n"
            if token_count + len(item_analysis) <= 5000:
                analysis += item_analysis
                token_count += len(item_analysis)
            else:
                break
    else:
        analysis += "\nNo upgrades or downgrades found for the specified date range.\n"

    # Handle Upgrades & Downgrades Consensus data dynamically
    if len(consensus_data) > 0:
        analysis += "\nUpgrades & Downgrades Consensus:\n"
        for item in consensus_data:
            for key, value in item.items():
                analysis += f"{key}: {value}\n"
            analysis += "\n"
    else:
        analysis += "\nNo upgrades and downgrades consensus found.\n"

    return analysis

def stock_price_history_tool(input: Annotated[StandardInput, "Input parameters for retrieving history of stock price data"]) -> str:
    ticker = input.ticker
    start_date = input.date_range[0]
    end_date = input.date_range[1]   

    stock_price_history_url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?from={start_date}&to={end_date}&apikey={FMP_API_KEY}" 
    stock_price_history_response = requests.get(stock_price_history_url)
    stock_price_history_data = stock_price_history_response.json()

    analysis = f"Stock Price History for {ticker} from {start_date} to {end_date}:\n\n"

    if stock_price_history_data is not None and "historical" in stock_price_history_data:
        for day_data in stock_price_history_data["historical"]:
            analysis += f"Date: {day_data['date']}\n"
            analysis += f"Open: {day_data['open']}\n"
            analysis += f"High: {day_data['high']}\n"
            analysis += f"Low: {day_data['low']}\n"
            analysis += f"Close: {day_data['close']}\n"
            analysis += f"Volume: {day_data['volume']}\n"
            analysis += "\n"
    else:
        analysis += "No data found for the specified date range.\n"

    return analysis        



def categorize_crypto_forex_stock_tool(input: Annotated[str, "Input prompt for classification"]) -> CategoryOutput:
    
    category = categorize_prompt_with_embeddings(input)
    return CategoryOutput(category=category)

# need to add the functionality when user prompt doesn't belongs to any of the category.
def categorize_crypto_forex_stock_tool_v2(input: Annotated[str, "Input prompt for classification"]) -> CategoryOutput:

    llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3
    )
    
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful assistant that categorizes the given user prompt to 'stock' or 'crypto', 'forex' or 'other' News.\
             Please respond with only the category name that is 'crypto' or 'forex' or 'stock' or 'other'. Please reply with only one word."),
            ("user", "{input}"),
            # ("instruction", parser.get_format_instructions()) 
        ]
    )
    
    chain = prompt | llm

    chain.invoke(
        {
            "input": input
        }
    )
    
    output = chain.invoke({"input": input})

    try:
        category_output = CategoryOutput(category=output.content)
    except Exception as e:
        raise ValueError(f"Failed to parse output: {output.content}. Error: {e}")

    return category_output


In [15]:
def categorize_crypto_forex_stock_tool_v3(user_prompt:str):
    start_time = time.time()
    skw,fkw,ckw = fetch_symbols()
    pkw = extract_keywords(user_prompt)
    category = identify_context(pkw,skw,fkw,ckw)

    if category == 'Unknown':
        print("Taking help of LLM..")
        category = categorize_crypto_forex_stock_tool_v2(user_prompt)
        total_time = time.time() - start_time  
        print(f"Total time taken: {total_time:.2f} seconds")
        return category.category
    else:
        if category == "Stock":
            skw.extend(pkw)
        if category == "Forex":
            fkw.extend(pkw)
        if category == "Crypto":
            ckw.extend(pkw)

        total_time = time.time() - start_time  
        print(f"Total time taken: {total_time:.2f} seconds")
        return category
    

In [16]:
def sec_insights_documents_selector(input: Annotated[str, "Qualitative data input prompt"]) -> List[Dict[str, str]]:

    llm = ChatOpenAI(
        model="gpt-4",
        temperature=0.3
    )
    
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", '''
                Your task is to identify and fill the following entities from the user's prompt:

                1. **ticker**: The abbreviation of the company's name.
                2. **document_type**: 'Form 10K' (for annual financial details), 'Form 10Q' (for quarterly financial details), etc.
                3. **year**: 
                   - If 'Form 10K' is chosen, the year should be in the format 'YYYY' (e.g., 2021, 2019).
                   - If 'Form 10Q' is chosen, the year should be in the format 'YYYY QX' (e.g., 2023 Q3, 2022 Q2).

                User prompts may be explicit or implicit and may not specify all required entities. Use context to infer missing details.

                If a user prompt is complex, you may break it into simpler, atomic prompts and provide separate outputs for each.
             
                Make a note that there is no fourth quarter Q4, companies file for Form 10K instead of filing Form 10Q. So you can only choose Q1, Q2 or Q3.

                ### Guidelines for Handling Qualitative Financial Questions:

                1. **Map to Relevant Documents**: Use the most relevant financial documents (annual or quarterly reports) that are likely to contain information on market trends, growth prospects, or other qualitative aspects.

                2. **Use Representative Entities**: When the user question is about a sector or general trends, use representative entities such as sector ETFs (e.g., XLK for technology) to provide the context.

                3. **Infer Contextual Details**: Use the context provided in the user's question to infer missing details and make reasonable assumptions about the tickers, document types and years.

                4. **Use top companies tickers**: If you are stuck which company's ticker to choose then use one or more top performing company's ticker in the respective field.
             
                5. **Use recent years**: If you are stuck with which year to choose then choose the latest year or years or latest quarter based on the context of the user prompt.

                ### Examples:
                1. **Prompt**: 'Please provide the report on recent developments at Apple.'
                   **Output**: 
                     ticker: AAPL
                     document_type: Form 10K
                     year: 2023
                     
                     ticker: AAPL
                     document_type: Form 10Q
                     year: 2023 Q3

                2. **Prompt**: 'Please provide the report on third quarter for Apple for the years 2021 and 2022.'
                   **Output**: 
                     ticker: AAPL
                     document_type: Form 10Q
                     year: 2021 Q3
                     
                     ticker: AAPL
                     document_type: Form 10Q
                     year: 2022 Q3

                3. **Prompt**: 'What are the prevailing market trends in the technology sector?'
                   **Output**: 
                     ticker: XLK
                     document_type: Form 10K
                     year: 2023
                     
                     ticker: XLK
                     document_type: Form 10Q
                     year: 2023 Q3

                4. **Prompt**: 'What are the growth prospects of Tesla in the next five years?'
                   **Output**: 
                     ticker: TSLA
                     document_type: Form 10K
                     year: 2023
                     
                     ticker: TSLA
                     document_type: Form 10Q
                     year: 2023 Q3

                Return 'TERMINATE' when the task is completed.
            '''),
            ("user", "{input}")
        ]
    )
    
    chain = prompt | llm
    
    output = chain.invoke({"input": input})

    try:
        documents = output.content.split('\n')
        result = []
        current_doc = {}

        for line in documents:
            if 'ticker:' in line:
                if current_doc:
                    result.append(current_doc)
                    current_doc = {}
                current_doc['ticker'] = line.split('ticker:')[1].strip()
            elif 'document_type:' in line:
                current_doc['document_type'] = line.split('document_type:')[1].strip()
            elif 'year:' in line:
                current_doc['year'] = line.split('year:')[1].strip()
        
        if current_doc:  # Add the last document
            result.append(current_doc)

        return result

    except Exception as e:
        raise ValueError(f"Failed to parse output: {output.content}. Error: {e}")



In [17]:
# Testing the function
input_prompt = "Please provide the report on performance of Apple and Microsoft for the years 2020 t0 2022"
try:
    sec_in_input = sec_insights_documents_selector(input_prompt)
    print(sec_in_input)
except ValueError as e:
    print(e)

[{'ticker': 'AAPL', 'document_type': 'Form 10K', 'year': '2020'}, {'ticker': 'AAPL', 'document_type': 'Form 10K', 'year': '2021'}, {'ticker': 'AAPL', 'document_type': 'Form 10K', 'year': '2022'}, {'ticker': 'MSFT', 'document_type': 'Form 10K', 'year': '2020'}, {'ticker': 'MSFT', 'document_type': 'Form 10K', 'year': '2021'}, {'ticker': 'MSFT', 'document_type': 'Form 10K', 'year': '2022'}, {'ticker': 'AAPL', 'document_type': 'Form 10Q', 'year': '2020 Q1'}, {'ticker': 'AAPL', 'document_type': 'Form 10Q', 'year': '2020 Q2'}, {'ticker': 'AAPL', 'document_type': 'Form 10Q', 'year': '2020 Q3'}, {'ticker': 'AAPL', 'document_type': 'Form 10Q', 'year': '2021 Q1'}, {'ticker': 'AAPL', 'document_type': 'Form 10Q', 'year': '2021 Q2'}, {'ticker': 'AAPL', 'document_type': 'Form 10Q', 'year': '2021 Q3'}, {'ticker': 'AAPL', 'document_type': 'Form 10Q', 'year': '2022 Q1'}, {'ticker': 'AAPL', 'document_type': 'Form 10Q', 'year': '2022 Q2'}, {'ticker': 'AAPL', 'document_type': 'Form 10Q', 'year': '2022 Q3'

In [18]:
categorize_crypto_forex_stock_tool("what is the current value of BTC")

CategoryOutput(category='forex')

In [19]:

categorize_crypto_forex_stock_tool_v2("what is eth?")

CategoryOutput(category='crypto')

In [20]:
categorize_crypto_forex_stock_tool_v3("AAPL")

Taking help of LLM..
Total time taken: 7.43 seconds


'stock'

In [21]:
# !python -m spacy download en_core_web_sm


In [22]:
system_message = """You are an expert financial analyst that always answers questions with the most relevant information using the tools at your disposal.
These tools have information regarding companies that the user has expressed interest in.
Here are some guidelines that you must follow:
* For financial questions, you must use the tools to find the answer and then write a response.
* Even if it seems like your tools won't be able to answer the question, you must still use them to find the most relevant information and insights. Not using them will appear as if you are not doing your job.
* You may assume that the users financial questions are related to the documents they've selected.
* For any user message that isn't related to financial analysis, respectfully decline to respond and suggest that the user ask a relevant question.
* If your tools are unable to find an answer, you should say that you haven't found an answer but still relay any useful information the tools found.

Provide a comprehensive answer to the user's question using the retrieved data. Dont just list data. If possible try to analyze the data and give interesting insights.

Return 'TERMINATE' when the task is completed.
"""

In [23]:
# Define the assistant agent
assistant = ConversableAgent(
    name="Assistant",
    system_message=system_message,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    max_consecutive_auto_reply=5,
    llm_config={"config_list": [{"model": "gpt-3.5-turbo-1106", "api_key": OPENAI_API_KEY}]},
    human_input_mode="NEVER",
)

# Define the user proxy agent
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)


In [24]:
# Register the upgrades and downgrades analysis tool
register_function(
    upgrades_downgrades_analysis_tool,
    caller=assistant,
    executor=user_proxy,
    name="upgrades_downgrades_analysis_tool",
    description="Generate an analysis of upgrades and downgrades for a given ticker and date range."
)

# Register the stock price history tool
register_function(
    stock_price_history_tool,
    caller=assistant,
    executor=user_proxy,
    name="stock_price_history_tool",
    description="Generate a historical stock price analysis for a given ticker and date range."
)

# Register the stock, forex and crypto news categorizer tool
register_function(
    categorize_crypto_forex_stock_tool_v2,
    caller=assistant,
    executor=user_proxy,
    name="categorize_crypto_forex_stock_tool_v2",
    description="Given a user prompt this tool helps in identifying whether it is a stock, crypto, forex or other news."
)

# Register the sec-insights input generator tool
register_function(
    sec_insights_documents_selector,
    caller=assistant,
    executor=user_proxy,
    name="sec_insights_documents_selector",
    description="Given a user prompt this tool helps to fill out the entities ticker,document_type and year which are qualified inputs for sec-insigths.ai application  "
)

In [25]:
# Example of using the tool in a conversation
chat_result = user_proxy.initiate_chat(
    assistant,
    message="Provide an analysis of upgrades and downgrades for AAPL from 2010-01-01 to 2023-12-31."
)

User (to Assistant):

Provide an analysis of upgrades and downgrades for AAPL from 2010-01-01 to 2023-12-31.

--------------------------------------------------------------------------------
Assistant (to User):

***** Suggested tool call (call_L69efqkLAlvekqjjYy4JGM4M): upgrades_downgrades_analysis_tool *****
Arguments: 
{"input":{"ticker":"AAPL","date_range":["2010-01-01","2023-12-31"]}}
**************************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION upgrades_downgrades_analysis_tool...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_L69efqkLAlvekqjjYy4JGM4M) *****
Stock Ticker: AAPL


Upgrades & Downgrades (Most Recent):
symbol: AAPL
publishedDate: 2023-12-18T13:18:00.000Z
newsURL: https://www.benzinga.com/news/23/12/36287592/apple-set-for-unprecedented-growth-analyst-predicts-stellar-rise-amid-market-tur

In [26]:
# Example of using the tool in a conversation
chat_result = user_proxy.initiate_chat(
    assistant,
    message="Provide the stock price history for MSFT from 2023-01-01 to 2023-12-31."
)

User (to Assistant):

Provide the stock price history for MSFT from 2023-01-01 to 2023-12-31.

--------------------------------------------------------------------------------
Assistant (to User):

***** Suggested tool call (call_ve8DGTKMenzyLgeMSqDsPdfm): stock_price_history_tool *****
Arguments: 
{"input":{"ticker":"MSFT","date_range":["2023-01-01","2023-12-31"]}}
*****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION stock_price_history_tool...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_ve8DGTKMenzyLgeMSqDsPdfm) *****
Stock Price History for MSFT from 2023-01-01 to 2023-12-31:

Date: 2023-12-29
Open: 376
High: 377.16
Low: 373.48
Close: 376.04
Volume: 18730838

Date: 2023-12-28
Open: 375.37
High: 376.46
Low: 374.16
Close: 375.28
Volume: 14327013

Date: 2023-12-27
Open: 373.69
High: 375.06
Low: 372.81


In [27]:
chat_result = user_proxy.initiate_chat(
    assistant,
    message="Can you tell me which category of news does SOLANA belongs to?"
)

User (to Assistant):

Can you tell me which category of news does SOLANA belongs to?

--------------------------------------------------------------------------------
Assistant (to User):

***** Suggested tool call (call_EbTxNoB1IEw9sQMXO68Rakdg): categorize_crypto_forex_stock_tool_v2 *****
Arguments: 
{"input":"SOLANA"}
******************************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION categorize_crypto_forex_stock_tool_v2...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_EbTxNoB1IEw9sQMXO68Rakdg) *****
{"category":"crypto"}
**********************************************************************

--------------------------------------------------------------------------------
Assistant (to User):

SOLANA belongs to the category of crypto news. If you have any specific questions related to SOLANA or any oth

In [28]:
chat_result = user_proxy.initiate_chat(
    assistant,
    message="How has the amazon's revenue and profit margin changed over the last five years?"
)

User (to Assistant):

How has the amazon's revenue and profit margin changed over the last five years?

--------------------------------------------------------------------------------
Assistant (to User):

***** Suggested tool call (call_yXJq04Jq7OHei7H3f19WSD3H): sec_insights_documents_selector *****
Arguments: 
{"input":"I am looking for Amazon's revenue and profit margin data over the last five years."}
************************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION sec_insights_documents_selector...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_yXJq04Jq7OHei7H3f19WSD3H) *****
[{"ticker": "AMZN", "document_type": "Form 10K", "year": "2019"}, {"ticker": "AMZN", "document_type": "Form 10K", "year": "2020"}, {"ticker": "AMZN", "document_type": "Form 10K", "year": "2021"}, {"ticker": "AMZN", "document_type":

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 93111 tokens (92858 in the messages, 253 in the functions). Please reduce the length of the messages or functions.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}